In [12]:
import networkx as nx

# 1. Supergraph 초기화
G_super = nx.DiGraph()

# 2. 4x4 걷기 격자 구축 (1칸 = 300m, 4.8km/h → 약 0.0625시간)
rows, cols = 4, 4
for i in range(rows):
    for j in range(cols):
        node = f"{chr(65+i)}{j+1}"
        if j < cols - 1:
            right = f"{chr(65+i)}{j+2}"
            G_super.add_edge(node, right, mode='walk', free_time=0.0625, capacity=999, flow=0)
        if i < rows - 1:
            down = f"{chr(65+i+1)}{j+1}"
            G_super.add_edge(node, down, mode='walk', free_time=0.0625, capacity=999, flow=0)

# 3. 버스 노선 (루프형)
bus_stops = ['B1', 'B2', 'B3', 'B4', 'C4', 'D4', 'D3', 'D2', 'D1', 'C1']
for u, v in zip(bus_stops, bus_stops[1:] + [bus_stops[0]]):
    G_super.add_edge(u, v, mode='bus', free_time=0.35, capacity=50, flow=0)

# 4. 메트로 노선 (장거리, 대각선)
metro_links = [('D1', 'B2'), ('B2', 'A4')]
for u, v in metro_links:
    G_super.add_edge(u, v, mode='metro', free_time=0.2, capacity=100, flow=0)

# 평균 시간 값 설정
# 걷기 → 버스 환승 시간
transfer_bus_time = (0.5 + 5 + 0.5) / 60  # 이동+대기+진입 시간 (min → hr)

# 걷기 → 메트로 환승 시간
transfer_metro_time = (0.5 + 4 + 0.5) / 60

# 환승 노드 정의
bus_transfer_nodes = set(bus_stops) - {'A4'}  # A4는 버스 정류장 아님
metro_transfer_nodes = {'D1', 'B2', 'A4'}

# 환승 링크 추가
for node in bus_transfer_nodes:
    G_super.add_edge(node, node, mode='transfer', free_time=transfer_bus_time, capacity=999, flow=0)

for node in metro_transfer_nodes:
    G_super.add_edge(node, node, mode='transfer', free_time=transfer_metro_time, capacity=999, flow=0)

In [13]:
import itertools

# OD 쌍 정의
nodes = [f"{chr(65+i)}{j+1}" for i in range(4) for j in range(4)]
OD_pairs = [(o, d) for o in nodes for d in nodes if o != d]

# 경로 비용 계산 함수
def compute_path_cost(G, path):
    cost = 0
    for i in range(len(path) - 1):
        if G.has_edge(path[i], path[i+1]):
            cost += G[path[i]][path[i+1]]['free_time']
        else:
            return float('inf')  # invalid path
    return cost

In [14]:
from collections import defaultdict

OD_best_routes = {}
OD_all_routes = defaultdict(list)
max_path_length = 10  # simple path 제한

for (origin, destination) in OD_pairs:
    all_routes = list(itertools.islice(nx.all_simple_paths(G_super, origin, destination, cutoff=max_path_length), 200))
    best_route = None
    best_cost = float('inf')

    for path in all_routes:
        cost = compute_path_cost(G_super, path)
        OD_all_routes[(origin, destination)].append((path, cost))
        if cost < best_cost:
            best_cost = cost
            best_route = path

    if best_route:
        OD_best_routes[(origin, destination)] = (best_route, best_cost)

In [15]:
# 초기 flow 0으로 설정
for u, v in G_super.edges:
    G_super[u][v]['flow'] = 0

# 수요 설정 (OD당 100명)
OD_demand_value = 100
OD_demand = {od: OD_demand_value for od in OD_best_routes}

# BPR 함수 정의
def bpr_cost(free_time, flow, capacity, alpha=0.15, beta=4):
    return free_time * (1 + alpha * (flow / capacity) ** beta)

# UE 반복
max_iter = 20
tolerance = 1e-3

for it in range(max_iter):
    link_cost = {}
    for u, v in G_super.edges:
        data = G_super[u][v]
        link_cost[(u, v)] = bpr_cost(data['free_time'], data['flow'], data['capacity'])

    od_flow = {e: 0 for e in G_super.edges}
    for (o, d), demand in OD_demand.items():
        try:
            shortest_path = nx.shortest_path(G_super, o, d, weight=lambda u, v, d: link_cost[(u, v)])
        except nx.NetworkXNoPath:
            continue
        for i in range(len(shortest_path) - 1):
            od_flow[(shortest_path[i], shortest_path[i+1])] += demand

    max_diff = 0
    for u, v in G_super.edges:
        old_flow = G_super[u][v]['flow']
        new_flow = 0.5 * old_flow + 0.5 * od_flow[(u, v)]
        G_super[u][v]['flow'] = new_flow
        max_diff = max(max_diff, abs(new_flow - old_flow))

    if max_diff < tolerance:
        print(f"UE converged at iteration {it}")
        break

In [16]:
import pandas as pd

flow_summary = []
for u, v in G_super.edges:
    data = G_super[u][v]
    flow_summary.append({
        'from': u,
        'to': v,
        'mode': data['mode'],
        'flow': round(data['flow'], 2),
        'final_cost': round(bpr_cost(data['free_time'], data['flow'], data['capacity']), 4)
    })

pd.DataFrame(flow_summary)

,from,to,mode,flow,final_cost
0,A1,A2,walk,600.00,6.370000e-02
1,A1,B1,walk,900.00,6.870000e-02
2,A2,A3,walk,1200.00,8.200000e-02
3,A2,B2,walk,700.00,6.480000e-02
4,B1,B2,bus,800.00,3.440977e+03
5,B1,C1,walk,1600.00,1.242000e-01
6,B1,B1,transfer,0.00,1.000000e-01
7,A3,A4,walk,600.00,6.370000e-02
8,A3,B3,walk,1700.00,1.411000e-01
9,B2,B3,bus,1200.00,1.741852e+04


In [17]:
# 총 시스템 cost 계산
# surviving OD쌍만 고려 (우리는 아직 실패 OD는 구현하지 않았음)
total_travel_cost = 0

for (o, d), demand in OD_demand.items():
    try:
        # 현재 링크 cost 기준 최단 경로
        path = nx.shortest_path(G_super, o, d, weight=lambda u, v, d: bpr_cost(
            G_super[u][v]['free_time'],
            G_super[u][v]['flow'],
            G_super[u][v]['capacity']
        ))
        # 경로 비용 계산
        cost = compute_path_cost(G_super, path)
        total_travel_cost += demand * cost
    except nx.NetworkXNoPath:
        continue  # 연결 불가능한 OD는 무시 (향후 penalty로 반영 가능)

print("Total system travel cost (sum over all OD):", round(total_travel_cost, 2))

Total system travel cost (sum over all OD): 14661.25
